In [1]:
import os
os.chdir('/mnt/mint_files/TCC/main_code/')

In [2]:
import numpy as np
from tensorflow import keras
from glob import glob
import rasterio as rio
from pyrsgis import raster
from pyrsgis.convert import changeDimension
from pyrsgis.convert import array_to_table
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from PIL import Image as im

2023-06-05 03:01:05.386104: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-06-05 03:01:05.386125: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [13]:
recreioFile = 'images/lagoa_freitas/2015/lagoa_freitas_v1_multiband_6.TIF'
# recreioFile = 'images/lagoa_freitas/land_5/2011/lagoa_freitas_v1_multiband_6.tif'
recreioDs, recreioFeatures = raster.read(recreioFile, bands='all')
print("Recreio multispectral image shape: ", recreioFeatures.shape)

Recreio multispectral image shape:  (6, 360, 500)


In [14]:
featuresRecreio = array_to_table(recreioFeatures) #changeDimension(recreioFeatures)
featuresRecreio.shape

(180000, 6)

In [5]:
np.min(featuresRecreio) 

5110

In [30]:
valoresCopiados = np.copy(featuresRecreio)
valMin =  np.min(featuresRecreio)
valorMax = np.max(featuresRecreio)
valoresCopiados = valoresCopiados - valMin
valoresCopiados = valoresCopiados / (valorMax - valMin)

valoresCopiados[0]

array([0.25603261, 0.22211957, 0.18494565, 0.1726087 , 0.30266304,
       0.27777174])

In [15]:
# Normalize Recreio
featuresRecreio = featuresRecreio / 65535.0 # 16 bits
# featuresRecreio = featuresRecreio / 255.0 # 8 bits

# Normaliza com a formula (x - x_min) / (x_max - x_min)
#valMin =  np.min(featuresRecreio)
#valorMax = np.max(featuresRecreio)
#featuresRecreio = featuresRecreio - valMin
#featuresRecreio = featuresRecreio / (valorMax - valMin)

featuresRecreio

array([[0.14985885, 0.14033722, 0.12990005, 0.12643626, 0.16295109,
        0.15596246],
       [0.15553521, 0.14813458, 0.13914702, 0.13800259, 0.16768139,
        0.17068742],
       [0.15364309, 0.14540322, 0.13670558, 0.13551537, 0.17149615,
        0.17833219],
       ...,
       [0.15686275, 0.14177157, 0.12680247, 0.11462577, 0.10626383,
        0.09524681],
       [0.15719844, 0.142916  , 0.12765698, 0.11599908, 0.10768292,
        0.09710842],
       [0.1588922 , 0.14421302, 0.12816052, 0.11584649, 0.1082475 ,
        0.09922942]])

In [16]:
# reshape
featuresRecreio = featuresRecreio.reshape((featuresRecreio.shape[0], 1, featuresRecreio.shape[1]))
print(featuresRecreio.shape)

(180000, 1, 6)


In [17]:
# Escolhe as bandas para serem utilizadas. entre 0 e 5. alterar a quantidade na primeira camada da RN
# [1,2,3] -> bands 2,3,4 - RGB
# [2,3,4] -> bands 3,4,5 - Infra vermelho
# [1,4,5] -> bands 2,5,6 - Agricultura
# [0,2,3] -> bands 1,3,4 - Bathymetric
# featuresRecreio_b = featuresRecreio[:,:,[1,2,3]]
print(np.min(featuresRecreio))
print(np.max(featuresRecreio))

0.07797360189211872
0.3587396047913329


In [20]:
valor_cortes = [0.005,0.01, 0.05, 0.10, 0.20, 0.30, 0.40, 0.50, 0.60, 0.70, 0.80, 0.90, 0.95, 0.99, 0.999, 0.9995, 0.9999]
# valor_cortes = [0.995, 0.999, 0.9995, 0.9999]
total_cortes = 5
arr_ep = ['2', '5', '10', '20', '25']
# arr_ep = ['2']
# arr_ep = ['100']

# land8
featuresRecreio_many = [
    ["234", np.copy(featuresRecreio[:,:,[1,2,3]])],
    ["345", np.copy(featuresRecreio[:,:,[2,3,4]])],
    ["256", np.copy(featuresRecreio[:,:,[1,4,5]])],
    ["134", np.copy(featuresRecreio[:,:,[0,2,3]])],
    ["allb", np.copy(featuresRecreio)]
]

# landsat_5
#featuresRecreio_many = [
#    ["234", np.copy(featuresRecreio[:,:,[0,1,2]])],
#    ["345", np.copy(featuresRecreio[:,:,[1,2,3]])],
#    ["256", np.copy(featuresRecreio[:,:,[0,3,4]])],
#    ["allb", np.copy(featuresRecreio)]
#]

# featuresRecreio_many = [["allb", featuresRecreio]]
total_features = 5
# 
for ep in arr_ep:
    for i in range(total_features):
        print(f'modelos/my_model_{featuresRecreio_many[i][0]}_{ep}.h5')
        modelo = keras.models.load_model(f'modelos/my_model_{featuresRecreio_many[i][0]}_{ep}.h5')
        predicao = modelo.predict(featuresRecreio_many[i][1])
        visualiza = predicao[:,1]
        for valCorte in valor_cortes:
            resultPredict = (visualiza>valCorte).astype(int)
            arrayOfnumeros = np.array(resultPredict)
            # (360, 500) - lagoa v1
            # (200, 150) - lagoa v2
            # (300, 650) - barra
            arrayOfnumeros = np.reshape(arrayOfnumeros, (360, 500))
            data = im.fromarray((arrayOfnumeros*255).astype(np.uint8))
            data.save(f'images/output_2_land8/{ep}_epochs/b_{featuresRecreio_many[i][0]}_{str(valCorte).replace(".", "_")}.png')

modelos/my_model_234_2.h5
modelos/my_model_345_2.h5
modelos/my_model_256_2.h5
modelos/my_model_134_2.h5
modelos/my_model_allb_2.h5
modelos/my_model_234_5.h5
modelos/my_model_345_5.h5
modelos/my_model_256_5.h5
modelos/my_model_134_5.h5
modelos/my_model_allb_5.h5
modelos/my_model_234_10.h5
modelos/my_model_345_10.h5
modelos/my_model_256_10.h5
modelos/my_model_134_10.h5
modelos/my_model_allb_10.h5
modelos/my_model_234_20.h5
modelos/my_model_345_20.h5
modelos/my_model_256_20.h5
modelos/my_model_134_20.h5
modelos/my_model_allb_20.h5
modelos/my_model_234_25.h5
modelos/my_model_345_25.h5
modelos/my_model_256_25.h5
modelos/my_model_134_25.h5
modelos/my_model_allb_25.h5
